In [4]:

import cufflinks as cf
import pandas as pd
import numpy as np
cf.set_config_file(offline=True)

In [5]:
rating_path = '/Users/xuanxinfang/Dropbox/UNSW/T2/COMP9417/assignment/dataset/rating.csv'
movie_path = '/Users/xuanxinfang/Dropbox/UNSW/T2/COMP9417/assignment/dataset/movie.csv'
tag_path = '/Users/xuanxinfang/Dropbox/UNSW/T2/COMP9417/assignment/dataset/tag.csv'
link_path = '/Users/xuanxinfang/Dropbox/UNSW/T2/COMP9417/assignment/dataset/link.csv'

ratings = pd.read_csv(rating_path)
# print(rating.head(10))
movies = pd.read_csv(movie_path)
# print(movies.head(10))

In [6]:
data = pd.merge(movies,ratings,on = 'movieId')
# print(data.head(5))
data[['userId','rating','movieId','title']].sort_values('userId').to_csv('data.csv',index=False)
# 将合并后的数据集输出保存到桌面 以备后续分析
file = pd.read_csv('data.csv')
print(file.head(5))

   userId  rating  movieId                       title
0       1     4.0        1            Toy Story (1995)
1       1     4.0     1777  Wedding Singer, The (1998)
2       1     5.0     1732    Big Lebowski, The (1998)
3       1     3.0     1676    Starship Troopers (1997)
4       1     5.0       50  Usual Suspects, The (1995)


In [9]:
file.iplot(kind='histogram', x="userid", y="rating")

In [11]:
cf.datagen.lines(1,500).ta_plot(study='sma',periods=[13,21,55])